In [46]:
import pandas as pd
import numpy as np
import config
import os
import statsmodels.api as sm

from get_data import DataLoader


DataLoader = DataLoader(config.DATA_PATH)
df = DataLoader.load_data()
df.describe()

,pid,wbc,age,chol,crp,mg
count,2350.000000,2350.000000,2350.000000,2350.000000,2350.000000,2350.000000
mean,1175.500000,11.564626,56.234468,152.042979,10.676881,0.814340
std,678.530889,16.645153,18.240480,55.011941,9.665577,0.147924
min,1.000000,0.010000,16.000000,27.000000,0.010000,0.200000
25%,588.250000,6.872500,42.250000,115.000000,2.572500,0.730000
50%,1175.500000,9.770000,58.000000,148.000000,8.120000,0.810000
75%,1762.750000,13.645000,69.000000,182.000000,16.067500,0.890000
max,2350.000000,604.470000,99.000000,646.000000,76.320000,1.880000


In [47]:
# Set seed with my birthday (yyyymmdd format)
seed = 19870909  

# Randomly sample 70% of the data
sample_df = df.sample(frac=0.7, random_state=seed)

# Generate summary statistics
table1 = sample_df.describe()
table1

,pid,wbc,age,chol,crp,mg
count,1645.000000,1645.000000,1645.000000,1645.000000,1645.000000,1645.000000
mean,1182.347112,11.679502,56.289362,152.586626,10.797258,0.814292
std,684.783767,18.883565,18.234687,56.271008,9.815020,0.148595
min,2.000000,0.010000,17.000000,28.000000,0.010000,0.290000
25%,583.000000,6.840000,43.000000,115.000000,2.510000,0.730000
50%,1192.000000,9.750000,58.000000,148.000000,8.320000,0.810000
75%,1775.000000,13.570000,69.000000,182.000000,16.320000,0.890000
max,2349.000000,604.470000,97.000000,646.000000,76.320000,1.880000


In [48]:
categorical_summary = {}

for col in sample_df.select_dtypes(include=['object', 'category']).columns:
    categorical_summary[col] = sample_df[col].value_counts()

# categorical_summary
cat_table = sample_df['sex'].value_counts().reset_index()
cat_table.columns = ['Category', 'Count']
cat_table['Percent'] = (cat_table['Count'] / cat_table['Count'].sum() * 100).round(1)
cat_table

,Category,Count,Percent
0,male,956,58.1
1,female,689,41.9


In [49]:
sample_df['sexFemale'] = (sample_df['sex'] == 'female').astype(int)
sample_df.head(5)

,pid,wbc,age,sex,chol,crp,mg,sexFemale
490,491,9.88,63,male,143,14.20,0.89,0
1136,1137,14.17,57,female,181,32.90,0.98,1
330,331,7.10,56,female,242,0.67,0.88,1
1091,1092,12.06,61,male,147,27.26,0.82,0
1480,1481,13.61,22,female,153,18.20,0.86,1


In [50]:
# Outcome
y = sample_df["wbc"]

# Covariates
covariates = ["age", "chol", "crp", "mg", "sexFemale"]

results = {}

# Single variable models
for cov in covariates:
    X = sm.add_constant(sample_df[cov])  # intercept
    model = sm.OLS(y, X).fit()
    results[cov] = model
    print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    wbc   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.4186
Date:                Tue, 26 Aug 2025   Prob (F-statistic):              0.518
Time:                        06:15:31   Log-Likelihood:                -7166.9
No. Observations:                1645   AIC:                         1.434e+04
Df Residuals:                    1643   BIC:                         1.435e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         10.7492      1.511      7.112      0.0

In [51]:
# Multivariable model
X_multi = sm.add_constant(sample_df[covariates])
model_multi = sm.OLS(y, X_multi).fit()
results["multivariable"] = model_multi
print(model_multi.summary())

                            OLS Regression Results                            
Dep. Variable:                    wbc   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     3.946
Date:                Tue, 26 Aug 2025   Prob (F-statistic):            0.00147
Time:                        06:15:31   Log-Likelihood:                -7157.3
No. Observations:                1645   AIC:                         1.433e+04
Df Residuals:                    1639   BIC:                         1.436e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.4123      3.309      1.938      0.0

In [ ]:
print(model_multi.summary())

# LATEX CODE

## Question 1

In [52]:
table1_latex = table1.to_latex(float_format="%.2f")
print(table1_latex)

\begin{tabular}{lrrrrrr}
\toprule
 & pid & wbc & age & chol & crp & mg \\
\midrule
count & 1645.00 & 1645.00 & 1645.00 & 1645.00 & 1645.00 & 1645.00 \\
mean & 1182.35 & 11.68 & 56.29 & 152.59 & 10.80 & 0.81 \\
std & 684.78 & 18.88 & 18.23 & 56.27 & 9.82 & 0.15 \\
min & 2.00 & 0.01 & 17.00 & 28.00 & 0.01 & 0.29 \\
25% & 583.00 & 6.84 & 43.00 & 115.00 & 2.51 & 0.73 \\
50% & 1192.00 & 9.75 & 58.00 & 148.00 & 8.32 & 0.81 \\
75% & 1775.00 & 13.57 & 69.00 & 182.00 & 16.32 & 0.89 \\
max & 2349.00 & 604.47 & 97.00 & 646.00 & 76.32 & 1.88 \\
\bottomrule
\end{tabular}



In [53]:
table1_cat_latex = cat_table.to_latex(index=False)
print(table1_cat_latex)

\begin{tabular}{lrr}
\toprule
Category & Count & Percent \\
\midrule
male & 956 & 58.100000 \\
female & 689 & 41.900000 \\
\bottomrule
\end{tabular}



## Question 2

In [57]:
table_rows = []

for name, model in results.items():
    summary_frame = model.summary2().tables[1]  # coeff table
    for var in summary_frame.index:
        if var == "const":  # skip intercept
            continue
        # intercept = summary_frame.loc[var, "const"]
        coef = summary_frame.loc[var, "Coef."]
        ci_low, ci_high = summary_frame.loc[var, ["[0.025", "0.975]"]]
        pval = summary_frame.loc[var, "P>|t|"]

        table_rows.append({
            "Model": name,
            "Variable": var,
            "Coef ($\beta$)": f"{coef:.3f}",
            "95% CI": f"[{ci_low:.3f}, {ci_high:.3f}]",
            "p-value": f"{pval:.3f}"
        })

# Add model fit stats for multivariable model
fit_stats = {
    "Model": "multivariable",
    "Variable": "Model fit",
    "Coef ($\beta$)": f"$R^2$={model_multi.rsquared:.3f}, Adj $R^2$={model_multi.rsquared_adj:.3f}, F={model_multi.fvalue:.3f}, $p$={model_multi.f_pvalue:.3g}",
    "95% CI": "",
    "p-value": ""
}
table_rows.append(fit_stats)

# Create dataframe
table2 = pd.DataFrame(table_rows)

table2_latex = table2.to_latex(index=False, escape=False)
print(table2_latex)

\begin{tabular}{lllll}
\toprule
Model & Variable & Coef (eta$) & 95% CI & p-value \\
\midrule
age & age & 0.017 & [-0.034, 0.067] & 0.518 \\
chol & chol & -0.009 & [-0.025, 0.007] & 0.273 \\
crp & crp & 0.200 & [0.108, 0.293] & 0.000 \\
mg & mg & 4.969 & [-1.175, 11.114] & 0.113 \\
sexFemale & sexFemale & -0.719 & [-2.570, 1.133] & 0.446 \\
multivariable & age & 0.005 & [-0.045, 0.055] & 0.854 \\
multivariable & chol & 0.002 & [-0.015, 0.019] & 0.794 \\
multivariable & crp & 0.196 & [0.098, 0.294] & 0.000 \\
multivariable & mg & 3.452 & [-2.711, 9.615] & 0.272 \\
multivariable & sexFemale & -0.661 & [-2.520, 1.199] & 0.486 \\
multivariable & Model fit & $R^2$=0.012, Adj $R^2$=0.009, F=3.946, $p$=0.00147 &  &  \\
\bottomrule
\end{tabular}

